<a href="https://colab.research.google.com/github/cala0/ttp/blob/master/terada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# doc2vecで寺田寅彦先生の随筆レコメンド

##0. 準備

###Google Driveと連携

In [0]:
from google.colab import drive
drive.mount('/content/drive')

###時刻をJSTに

★　環境変数TZの内容は別セルには引き継がれない。.bashrcに書き込んでも、読んでくれない。

In [0]:
%%bash

echo "export TZ='Asia/Tokyo'" > .bashrc
source .bashrc
ls -la

total 4036
drwxr-xr-x 1 root root    4096 May  5 21:15 .
drwxr-xr-x 1 root root    4096 May  5 14:12 ..
-rw-r--r-- 1 root root      23 May  5 21:43 .bashrc
drwxr-xr-x 1 root root    4096 Apr 30 01:32 .config
-rw-r--r-- 1 root root   24420 May  5 18:57 doc_list.txt
-rw-r--r-- 1 root root    5283 May  5 15:18 doc_name.txt
-rw-r--r-- 1 root root 4040395 May  5 15:18 doc_text.txt
drwx------ 3 root root    4096 May  5 14:15 drive
-rw-r--r-- 1 root root   11809 May  5 14:35 res.txt
drwxr-xr-x 1 root root    4096 Apr 30 01:32 sample_data
drwxrwx--- 2 root  999   20480 Apr 21 23:54 terada


In [0]:
%%bash

ls -la

total 4036
drwxr-xr-x 1 root root    4096 May  5 12:15 .
drwxr-xr-x 1 root root    4096 May  5 05:12 ..
-rw-r--r-- 1 root root      23 May  5 12:43 .bashrc
drwxr-xr-x 1 root root    4096 Apr 29 16:32 .config
-rw-r--r-- 1 root root   24420 May  5 09:57 doc_list.txt
-rw-r--r-- 1 root root    5283 May  5 06:18 doc_name.txt
-rw-r--r-- 1 root root 4040395 May  5 06:18 doc_text.txt
drwx------ 3 root root    4096 May  5 05:15 drive
-rw-r--r-- 1 root root   11809 May  5 05:35 res.txt
drwxr-xr-x 1 root root    4096 Apr 29 16:32 sample_data
drwxrwx--- 2 root  999   20480 Apr 21 14:54 terada


##1. 青空文庫からデータ収集

###メタデータの収集

In [0]:
#coding: UTF-8
import urllib.request, urllib.error  # for Python3 urllib2
from bs4 import BeautifulSoup
import time

urls = []
titles = []

# 作家別作品リスト（寺田寅彦）のページを開き、
list_page = urllib.request.urlopen('https://www.aozora.gr.jp/index_pages/person42.html')
list_soup = BeautifulSoup(list_page, "html.parser")

# 各作品の図書カードへのリンクから
for card in list_soup.select("body > ol > li > a"):

    # 作品タイトルと図書カードURLを取得
    title = card.text
    card_url = 'https://www.aozora.gr.jp/' + card.get('href')[3:]

    try:
        # 当該作品の図書カードのページを開き、
        card_page = urllib.request.urlopen(card_url)
        card_soup = BeautifulSoup(card_page, "html.parser")
    
        # 作品ダウンロードへのリンク（いますぐXHTML版で読む）から、作品URLを取得
        url = 'https://www.aozora.gr.jp/cards/000042/' + card_soup.select_one("body > div:nth-of-type(3) > a:nth-of-type(2)").get('href')[2:]
    
        # タイトルとURLをリストに記録
        urls.append(url)
        titles.append(title)
        print(title, url)
    
        time.sleep(1)
        
    except:
        print(title + '... error skip!')


In [0]:
# タイトルとURLのリストをファイルに書き出す
doc_list = open("doc_list.txt", "w")

for i in range(len(titles)):
    doc_list.write(str(i+1) + ', ' + titles[i] + ', ' + urls[i] + '\n')
    
doc_list.close()

★「宇宙の始まり」は、著者アレニウス スヴァンテ、訳者寺田寅彦なので、著作者IDが違う。000042ではなく、000226。

URL生成時に、000042決め打ちにしているため、誤ったURLになっている。

In [0]:
!cat doc_list.txt

In [0]:
%%bash
wc *.txt
cp *.txt /content/drive/'My Drive'/'Colab Notebooks'/TTP/

    288     869   24420 doc_list.txt
    286     286    5283 doc_name.txt
    286  572889 4040395 doc_text.txt
    286    3146   11809 res.txt
   1146  577190 4081907 total


###作品一括ダウンロード
###作品テキストの整形
*   SJISからUTF-8へ変換
*   ルビ、注記、ヘッダ、フッタ類を削除

###とりあえず、Google Driveから収集済の作品テキストを展開する

In [0]:
%cd /content
!ls
!tar zxvf /content/drive/'My Drive'/'Colab Notebooks'/TTP/terada.tar.gz .
!ls

##2. MeCabで形態素解析

###MeCabのインストール

In [0]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

In [0]:
!pip install mecab-python3

In [0]:
import MeCab

m = MeCab.Tagger ("-Ochasen")
print(m.parse ("すもももももももものうち"))

すもも	スモモ	すもも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
の	ノ	の	助詞-連体化		
うち	ウチ	うち	名詞-非自立-副詞可能		
EOS



###作品テキストを形態素解析

In [0]:
#coding: utf-8
import sys
import MeCab

#	mecabによる形態素解析：入力テキスト中の名詞、形容詞、動詞、副詞の標準表記のリストを返却する
def extractKeyword(text):
    tagger = MeCab.Tagger('-Ochasen')
    text = text.replace('\n', '')
#    text = text.encode('utf-8')
#    ★標準入力からUTF8のテキストファイルを読み込む場合、エラーになる
    node = tagger.parseToNode(text)
    keywords = []
    while node:
#        if node.feature.split(",")[0] == "名詞" and node.feature.split(",")[1] != "非自立":
        if node.feature.split(",")[0] in ["名詞", "動詞", "形容詞", "形容動詞"]:
#           print node.surface + "\t" + node.feature
            plain_word = node.feature.split(",")[6]
            if plain_word != "*":
                keywords.append(plain_word)
            else:
                keywords.append(node.surface)
#                keywords.append(node.surface.decode('utf-8'))
        node = node.next
    return keywords

In [0]:
from glob import glob

doc_text = open("doc_text.txt", "w")
doc_name = open("doc_name.txt", "w")

for file in sorted(glob('terada/*.txt')):
    f = open(file, "r")
    txt = f.read()
    f.close()
    kwds = extractKeyword(txt)    

    doc_text.write(' '.join(kwds) + '\n')
    doc_name.write(file.replace(dir_name+'/', '').replace('.txt', '') + '\n')
    
doc_text.close()
doc_name.close()

In [0]:
%%bash
head doc_name.txt
head doc_text.txt
wc doc_*.txt

Liber_Studiorum
PROFESSOR_TAKEMATU_OKADA
「手首」の問題
あひると猿
ある幻想曲の序
ある探偵事件
ある日の経験
からすうりの花と蛾
さまよえるユダヤ人の手記より
さるかに合戦と桃太郎
LIBER STUDIORUM 寺田 寅彦 震災 後 復興 第一歩 行なう れる 浅草 凌雲 閣 爆破 見物 行く 工兵 数 人 かかる 塔 もと 穴 うがつ いる 穴 爆薬 仕掛ける 一 度 倒壊 する せる の 倒れる 方向 定める ため 倒す する 方向 側面 穴 数 多い する いる 準備 整う 予定 時刻 迫る 見物人 ら 一定 距離 画す 非常 線 外 退去 命ずる られる 自分 ら 花屋敷 鉄 檻 裏手 焼け跡 行く 合図 ラッパ 鳴る の 待つ いる 時 一 匹 のら犬 トボトボ 人間 許す れる 警戒 線 越える 倒壊 する 塔 ほう こと 知る ひもじい そう 焼け跡 土 かぐ 近寄る 行く の 見える 塔 もと まっか 雲 八方 ほとばしる わく 上がる 思う 塔 十 二 階 三 四 片 折れ曲がる 折れ線 なる 次 瞬間 粉々 もむ 砕く れる よう なる 目 見える 漏斗 紅殻 色 灰 落とす よう 直下 堆積 する ステッキ 倒す よう 倒れる もの 皆 考える いる の 塔 一方 壁 サーベル 立てる よう 形 なる くずれる 残る の 弱い 爆発 きれい もむ 砕く しまう 爆破 言葉 こわれる 方 ふさわしい 今 堅い 岩 できる いる もの 土 灰 落雁 よう もの 変わる たれる 落ちる 思う れる もと ダイナマイト 付近 爆裂 する 二 三 百 メートル 距離 豌豆 大 煉瓦 破片 一つ 二つ 飛ぶ 来る 石垣 ぶつかる の 見る 爆破 瞬間 四方 はい出す まっか 雲 珍しい がめ 紅毛 唐獅子 百 匹 一 度 おどる 出す よう くずれる 終わる 見物人 一 度 押し寄せる 酔狂 二 三 人 たち 先 争う 砕ける 煉瓦 山 頂上 駆ける 上がる 中 バンザーイ 叫ぶ の いる よう 記憶 する 明治 煉瓦 時代 最後 守り よう 踏みとどまる いる 巨人 立つ 腹 切る 倒れる その後 来る もの 鉄筋コンクリート 時代 ジャズ トーキー プロ 文学 時代 時 塔 ほう 近づく 行く 小犬 する 

##3. doc2vecで類似文書検索

In [0]:
#coding: utf-8

from gensim.models import doc2vec
import codecs
import sys

# テキスト読み込み
docs = doc2vec.TaggedLineDocument('doc_text.txt')

# 名前読み込み（結果表示に使用する）
with codecs.open('doc_name.txt', 'r', encoding='utf-8') as f:    
    doc_names = [name.replace('\n', '') for name in f.readlines()]

# モデルデータ作成
# vector_size: ベクトル圧縮時の次元数
# window: 解析時に対象単語の前後何単語まで見るか
# min_count: 出現回数が何回以下の単語を無視するか
# workers: スレッド数
model = doc2vec.Doc2Vec(docs, vector_size=100, window=10, min_count=2, workers=4)

# データ保存
#model.save('d.model')
#model.save_word2vec_format('d.w2vmodel')

# 類似単語
#word = u'物理'
#for similar_word in model.most_similar(positive=word):
#	print similar_word[0], similar_word[1]

# 各文書について、類似度上位10件の文書リストを出力
f = open("res.txt", "w")
for idx in range(len(doc_names)):
    f.write(str(idx) + ' ')
    for similar_doc in model.docvecs.most_similar(idx):  
#       print(doc_names[idx].encode('utf-8'), doc_names[similar_doc[0]].encode('utf-8'), similar_doc[1])
#       print(doc_names[idx], doc_names[similar_doc[0]], similar_doc[1])
#	    print(similar_doc[0])
        f.write(str(similar_doc[0]) + ' ')
#    print('\n')
    f.write('\n')
f.close()

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
%%bash
tail res.txt
wc res.txt

276 106 109 270 122 221 265 50 227 96 75 
277 99 117 176 149 34 0 78 9 33 163 
278 175 43 161 132 76 31 111 81 88 162 
279 159 269 62 115 182 215 86 125 123 22 
280 211 108 176 117 0 20 118 53 149 99 
281 18 232 244 100 129 255 236 87 178 119 
282 27 183 4 67 229 105 36 167 55 45 
283 168 179 141 111 31 83 110 171 71 259 
284 69 258 265 193 201 270 188 109 134 276 
285 42 64 243 121 170 32 187 70 165 7 
  286  3146 11809 res.txt


In [0]:
%%bash
wc *.txt
cp *.txt /content/drive/'My Drive'/'Colab Notebooks'/TTP/

    286     286    5283 doc_name.txt
    286  344461 2286917 doc_text.txt
    286    3146   11809 res.txt
    858  347893 2304009 total


##4. レコメンドデータ生成

In [0]:
%%bash
ls -la

total 4008
drwxr-xr-x 1 root root    4096 May  5 05:32 .
drwxr-xr-x 1 root root    4096 May  5 05:12 ..
drwxr-xr-x 1 root root    4096 Apr 29 16:32 .config
-rw-r--r-- 1 root root    5283 May  5 06:18 doc_name.txt
-rw-r--r-- 1 root root 4040395 May  5 06:18 doc_text.txt
drwx------ 3 root root    4096 May  5 05:15 drive
-rw-r--r-- 1 root root   11809 May  5 05:35 res.txt
drwxr-xr-x 1 root root    4096 Apr 29 16:32 sample_data
drwxrwx--- 2 root  999   20480 Apr 21 14:54 terada
